In [27]:
# %pip install -U -q pydantic
from pydantic import BaseModel, Field
import json
import re
# %pip install -U -q pandas
import pandas as pd
# %pip install -U -q langchain_ollama
from langchain_ollama import ChatOllama
# %pip install -U -q langchain-openai
from langchain_openai import ChatOpenAI
# %pip install -U -q neo4j
from neo4j import GraphDatabase
#%pip install -U -q spacy
#import spacy



In [28]:
filename = "/Users/brianmann/Downloads/ori_pqal.json" #Change with your path
with open(filename, 'r', encoding='utf-8') as f:
    labeled_instances = json.load(f)

#get questions and long answers 
# IMPORTANT: THIS ASSUMES THAT EACH ENTRY IN THE JSON FILE HAS A QUESTION, CONCLUSION, GROUND TRUTH IN THIS FORMAT. 
# IF WE FIND BAD RESULTS IN LATER QUESTIONS LOOK HERE
questions = []
conclusions = []
ground_truth = []
for paper in labeled_instances:
    questions.append(labeled_instances[paper]['QUESTION'])
    conclusions.append(labeled_instances[paper]['LONG_ANSWER'])
    ground_truth.append(labeled_instances[paper]['final_decision'])

test_size = 50
# Only use first 5 for now
questions = questions[:test_size]
titles = questions[:test_size]
conclusions = conclusions[:test_size]
ground_truth = ground_truth[:test_size]

In [29]:
llm = ChatOllama(model="llama3.2", temperature=0, max_tokens=2048) 

# response = llm.invoke(structured_text)
# raw_output = response.content

In [30]:
def create_query(title: str, conclusion: str):
    query = f'''
    Using this information {conclusion}

    Answer the following quesiton with yes, no, or maybe.: {title} 
    
    '''
    return query

In [31]:
answers = [] #

for title, conclusion in zip(questions, conclusions):
    response = llm.invoke(create_query(title, conclusion))
    raw_output = response.content
    first_word = raw_output.split()[0].rstrip('.').lower()
    answers.append(first_word)
    # print(first_word)
    

In [32]:
sum([a ==b for a,b in zip(answers, ground_truth)])

39